In [1]:
import pandas as pd
from db_utils import get_engine

In [2]:
dataset_path = r"C:\Users\User\PycharmProjects\lufthansa_task\lufthansa_task\cleaned_dataset"

In [3]:
customers = pd.read_csv(dataset_path + '/customers.csv')
order_items = pd.read_csv(dataset_path +'/order_items.csv')
order_payments = pd.read_csv(dataset_path +'/order_payments.csv')
order_reviews = pd.read_csv(dataset_path +'/order_reviews.csv')
orders = pd.read_csv(dataset_path +'/orders.csv')
products = pd.read_csv(dataset_path +'/products.csv')
sellers = pd.read_csv(dataset_path +'/sellers.csv')
data = {'customers': customers,
        'order_items': order_items, 
        'order_payments': order_payments,
        'order_reviews': order_reviews,
        'orders': orders,
        'products': products,
        'sellers': sellers}

In [4]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110189 entries, 0 to 110188
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       110189 non-null  object 
 1   order_item_id                  110189 non-null  int64  
 2   product_id                     110189 non-null  object 
 3   seller_id                      110189 non-null  object 
 4   shipping_limit_date            110189 non-null  object 
 5   price                          110189 non-null  float64
 6   freight_value                  110189 non-null  float64
 7   total_price                    110189 non-null  float64
 8   order_purchase_timestamp       110189 non-null  object 
 9   order_delivered_customer_date  110189 non-null  object 
 10  delivery_time                  110189 non-null  int64  
 11  total_payments_count           110186 non-null  float64
 12  profit_margin                 

Fact order items table.

In [5]:
# Get the most important columns and also the calculated columns for the fact_order_items. Merge with orders to get the customer id we need for a query later.

fact_order_items = order_items[['order_id', 'order_item_id', 'product_id', 'seller_id', 'price', 'freight_value',
                                'total_price', 'delivery_time', 'total_payments_count', 'profit_margin']].merge(orders[['order_id', 'customer_id']], on='order_id', how='left')

        Customers Dimension

In [6]:
# Get the dimension columns for customers table
dim_customers = customers[['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
                           'customer_city', 'customer_state']]

Product Dimension

In [7]:
# Get the dimension columns for products table
dim_products = products[['product_id', 'product_category_name',
                         'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm',
                         'product_width_cm']]

Sellers Dimension

In [8]:
# Get the dimension columns for sellers table
dim_sellers = sellers[['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']]

Date Dimension

In [9]:
# Use order_purchase_timestamp and also order_delivered_customer_date for the date table. 
# First we want to set the purchase and delivery to datetime.  
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
dim_date = orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date']].copy()

In [10]:
engine = get_engine()

In [11]:
# Fact Table
fact_order_items.to_sql('fact_order_items', con=engine, if_exists='replace', index=False)

# Dimension Tables
dim_customers.to_sql('dim_customers', con=engine, if_exists='replace', index=False)
dim_products.to_sql('dim_products', con=engine, if_exists='replace', index=False)
dim_sellers.to_sql('dim_sellers', con=engine, if_exists='replace', index=False)
dim_date.to_sql('dim_date', con=engine, if_exists='replace', index=False)

8